__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-16 15:02:15 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-16 15:02:15 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-16 15:02:17 | AdsListParser: downloader init
[INFO    ] 2022-09-16 15:02:17 | CianParser: downloader init
[INFO    ] 2022-09-16 15:02:17 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-16 15:02:32 | AdsListParser: read page 1
[INFO    ] 2022-09-16 15:02:39 | AdsListParser: read page 2
[INFO    ] 2022-09-16 15:02:50 | AdsListParser: read page 3
[INFO    ] 2022-09-16 15:03:01 | AdsListParser: read page 4
[INFO    ] 2022-09-16 15:03:10 | AdsListParser: read page 5
[INFO    ] 2022-09-16 15:03:19 | AdsListParser: read page 6
[INFO    ] 2022-09-16 15:03:25 | AdsListParser: read page 7
[INFO    ] 2022-09-16 15:03:37 | AdsListParser: read page 8
[INFO    ] 2022-09-16 15:03:49 | AdsListParser: read page 9
[INFO    ] 2022-09-16 15:03:58 | AdsListParser: read page 10
[INFO    ] 2022-09-16 15:04:04 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
1400,"3-комн. кв., 115,3 м², 8/9 этаж",,,18 800 000 ₽,163 053 ₽/м²,"[Севастополь, р-н Гагаринский, улица Героев Бреста, 116]","[2 часа назад, сегодня, 12:39]",https://sevastopol.cian.ru/sale/flat/276987635/,"Продам элитную 3-х комнатную квартиру в лучшем районе города рядом с морем. В доме с закрытой территорией по адресу ул. Героев Бреста, дом 116. \n\n Квартира в идеальном состоянии, расположена на 8 этаже и имеет общую площадь 115,3 квадратных метра. Высота потолка 2.8 м. В этой квартире сделана очень грамотная планировка: две отдельные спальные комнаты; техническая комната, где находятся стиральная и сушильная машины, система водоподготовки и фильтрации воды, а также система рекуперации воздуха; просторная гостиная; кухня-столовая и, конечно же, гардеробная комната. Также установлена система видеонаблюдения. Безопасность вас и ваших близких будет обеспечена.\n\n В квартире сделан потрясающий ремонт с использованием только дорогих и качественных материалов, который вы сможете оценить только посмотрев ее. Установлена самая современная техника и мебель. Скажу только одно: в ремонт, технику и мебель вложено более 9 млн рублей. И все это останется вам. \n \n Я не буду рассказывать о преимуществах закрытой территории и о расположении дома, что в пешей доступности есть несколько пляжей. Вы это, наверняка, и сами знаете. Эта квартира идеально подчеркнет ваш статус.\n\n Если вы современный человек, если вы хотите проживать в одном из самых перспективных районов города, если вы цените современные решения и для вас важен простор?! Эта квартира для вас!\n\n Огромным плюсом будет вид из окна гостиной на бухту и море, а таких в Севастополе мало. Кстати, окна везде панорамные. А если вы захотите ограничить попадание света, в этом вам помогут шторы блэкаут, которые висят по всему периметру. \n\n Не упустите свой шанс стать собственником престижной недвижимости. \nЗвоните! Покупайте!",51,2022-09-16 15:11:49.899057
1226,"3-комн. кв., 46,2 м², 1/2 этаж",,,10 500 000 ₽,227 273 ₽/м²,"[Севастополь, р-н Ленинский, улица Супруна, 5]","[2 часа назад, сегодня, 12:38]",https://sevastopol.cian.ru/sale/flat/276346669/,"Трехкомнатная квартира 46,2 м2, на 1/2 этаже дома. Расположена в г. Севастополь, Ленинский район, ул. Супруна, д. 5.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Ленинский район, ул. Супруна, д. 5\n-Год постройки дома: 1950\n-Этаж/этажность: 1/2\n-Количество комнат: 3\n-Комнаты: изолированные\n-Общая площадь: 46,2 м2\n-Жилая площадь: 30,8 м2\n-Комната: 13,9; 11,3 и 5,6 м2\n-Площадь кухни: 7,3 м2\n-Высота потолка: 3,1 м\n-Санузел: совмещенный\n-Водоснабжение: центральное\n-Горячее водоснабжение: газовая колонка\n-Отопление: центральное\nДополнительная информация: \nКвартира общей площадью 46,2 м2, имеет три изолированные комнаты 13,9 м2, 11,3 м2, 5,6 м2. \nВ квартире сделан евроремонт, все мебель и техника остается. Имеется отдельный вход в подвальное помещение площадью более 10 м2.\nИмеется отдельный выход на огороженную террасу, на которой уложена плитка, что позволит свободно парковать машину. Дом оборудован системой видеонаблюдения. \nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nДом расположен в историческом центре города.\nВ шаговой доступности поликлиника 4, политехнический лицей, торгово-экономический техникум, школы 65, 45, 14, детский сад 2. До Исторического Бульвара пешком 5 минут. Возможность добраться на общественном транспорте в любую точку города. Рассматривается вариант обмена на дом в черте города.\nНаличный расчет подразумевают приличную скидку.\nДокументы РФ проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять!",44,2022-09-16 15:11:49.899057
645,"1-комн. кв., 20 м², 2/3 этаж",,,2 700 000 ₽,135 000 ₽/м²,"[Севастополь, р-н Гагаринский, улица Готская

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1477


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [10]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [11]:
# df

In [12]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [13]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [14]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])